# Anomaly Detection Project
By Jeff Akins

### ![](https://mk0crunchmetric1ct0g.kinstacdn.com/wp-content/uploads/2019/10/Anomaly-detection-on-time-series-data-with-auto-encoders.jpg)

In [1]:
import numpy as np
import pandas as pd
from datetime import datetime

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn import metrics
import prepare as prep

In [2]:
log = prep.prep_logs()

In [3]:
log.head()

,path,user_id,cohort_id,ip,name,slack,start_date,end_date,created_at,updated_at,program_id,course_name,course_subdomain,hour,weekday
timestamp,,,,,,,,,,,,,,,
2018-01-26 09:55:03,/,1,8,97.105.19.61,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,1,PHP Full Stack Web Dev,php,9,Friday
2018-01-26 09:56:02,java-ii,1,8,97.105.19.61,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,1,PHP Full Stack Web Dev,php,9,Friday
2018-01-26 09:56:05,java-ii/object-oriented-programming,1,8,97.105.19.61,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,1,PHP Full Stack Web Dev,php,9,Friday
2018-01-26 09:56:06,slides/object_oriented_programming,1,8,97.105.19.61,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,1,PHP Full Stack Web Dev,php,9,Friday
2018-01-26 09:56:24,javascript-i/conditionals,2,22,97.105.19.61,Teddy,#teddy,2018-01-08,2018-05-17,2018-01-08 13:59:10,2018-01-08 13:59:10,2,Java Full Stack Web Dev,java,9,Friday


In [4]:
log.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 900223 entries, 2018-01-26 09:55:03 to 2021-04-21 16:44:39
Data columns (total 15 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   path              900222 non-null  object        
 1   user_id           900223 non-null  int64         
 2   cohort_id         900223 non-null  int64         
 3   ip                900223 non-null  object        
 4   name              900223 non-null  object        
 5   slack             900223 non-null  object        
 6   start_date        900223 non-null  datetime64[ns]
 7   end_date          900223 non-null  datetime64[ns]
 8   created_at        900223 non-null  datetime64[ns]
 9   updated_at        900223 non-null  datetime64[ns]
 10  program_id        900223 non-null  int64         
 11  course_name       900223 non-null  object        
 12  course_subdomain  900223 non-null  object        
 13  hour              900223 

In [ ]:
log = prep.get_logs()
log.head()

In [ ]:
log.info()

In [ ]:
log.program_id.value_counts()

In [ ]:
course = pd.DataFrame({'id': [1, 2, 3, 4], 
                       'course_name': ['PHP Full Stack Web Dev', 'Java Full Stack Web Dev', 'Data Science', 'Front End Web Dev'],
                       'course_subdomain': ['php', 'java', 'ds', 'fe']})
course

In [ ]:
log = log.merge(course, how='left', left_on='program_id', right_on='id')

In [ ]:
log = log.drop(columns=['Unnamed: 0', 'id_y', 'id_x', 'deleted_at'])

In [ ]:
log.head()

In [ ]:
log.info()

In [ ]:
log.start_date = log.start_date.fillna(log['date'])
log.end_date = log.end_date.fillna(log['date'])
log.created_at = log.created_at.fillna(log['date'])
log.updated_at = log.updated_at.fillna(log['date'])

In [ ]:
log['timestamp'] = log.date + " " + log.time
log.timestamp = pd.to_datetime(log.timestamp)
log = log.set_index('timestamp')

log['hour'] = log.index.hour
log['weekday'] = log.index.day_name()

In [ ]:
log.start_date = pd.to_datetime(log.start_date)
log.end_date = pd.to_datetime(log.end_date)

In [ ]:
log.created_at = pd.to_datetime(log.created_at)
log.updated_at = pd.to_datetime(log.updated_at)

In [ ]:
log = log.drop(columns=['date', 'time'])

In [ ]:
log.cohort_id = log.cohort_id.fillna(0)
log.program_id = log.program_id.fillna(0)

log.cohort_id = log.cohort_id.astype('int64')
log.program_id = log.program_id.astype('int64')

In [ ]:
column_list = ['name', 'slack', 'course_name', 'course_subdomain']
for col in column_list:
    log[col] = log[col].fillna('none')

In [ ]:
log.head()

In [ ]:
log.info()

In [ ]:
log[log.start_date.isnull()]

In [ ]:
plt.figure(figsize=(12, 8))
sns.histplot(data=log,
    x="hour", hue="cohort_id",
    palette="ch:s=.25,rot=-.25",)

In [ ]:
df[~ df.path.str.endswith('jpeg')]